In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

# Define a simple CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Define data transforms and load CIFAR-100 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Initialize the network, loss function, and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Training loop
best_accuracy = 0
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {accuracy:.2f}%")

    # Save the model with the best validation accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(net.state_dict(), "best_model.pth")

print("Finished Training")

# Load the best model
net.load_state_dict(torch.load("best_model.pth"))

# Evaluate the best model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f"Testing Accuracy of the Best Model: {test_accuracy:.2f}%")


100%|██████████| 169001437/169001437 [00:02<00:00, 75742202.84it/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Loss: 4.144, Validation Accuracy: 13.58%
Epoch 2, Loss: 3.344, Validation Accuracy: 24.36%
Epoch 3, Loss: 2.863, Validation Accuracy: 31.14%
Epoch 4, Loss: 2.496, Validation Accuracy: 35.17%
Epoch 5, Loss: 2.187, Validation Accuracy: 39.10%
Epoch 6, Loss: 1.906, Validation Accuracy: 40.94%
Epoch 7, Loss: 1.629, Validation Accuracy: 42.14%
Epoch 8, Loss: 1.353, Validation Accuracy: 40.63%
Epoch 9, Loss: 1.093, Validation Accuracy: 41.06%
Epoch 10, Loss: 0.838, Validation Accuracy: 40.82%
Epoch 11, Loss: 0.635, Validation Accuracy: 40.53%
Epoch 12, Loss: 0.497, Validation Accuracy: 41.01%
Epoch 13, Loss: 0.414, Validation Accuracy: 40.03%
Epoch 14, Loss: 0.349, Validation Accuracy: 39.28%
Epoch 15, Loss: 0.294, Validation Accuracy: 40.02%
Epoch 16, Loss: 0.235, Validation Accuracy: 39.70%
Epoch 17, Loss: 0.214, Validation Accuracy: 40.49%
Epoch 18, Loss: 0.191, Validation Accuracy: 39.63%
E